In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 15  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_15_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma15/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:1'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k_new = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)
            rel_err_out = (torch.norm(c_k - c_k_new) / torch.norm(c_k)).item() 
            sc = "{:0.1e}".format(rel_err_out)
            c_k = c_k_new
    
    return c_k

In [6]:
i = 0
psnrs = list()

with torch.no_grad():
    for img in test_dataset:
        i = i + 1
        if True:
            gt = img.to(device)[None, :, :, :]
            noisy_image = (img.to(device) + (15/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
            denoised =safi_denoise(noisy_image)
            psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
            psnrs.append(psnr)
            print(i, psnr)

1 32.41
2 30.93
3 33.23
4 32.24
5 32.5
6 29.36
7 31.72
8 31.3
9 32.76
10 31.48
11 30.79
12 28.93
13 31.95
14 29.32
15 30.73
16 33.22
17 30.86
18 31.53
19 33.26
20 31.91
21 35.27
22 29.13
23 28.95
24 27.53
25 30.61
26 31.44
27 30.76
28 35.16
29 29.04
30 32.24
31 30.41
32 33.81
33 32.28
34 26.91
35 31.17
36 30.71
37 31.62
38 30.1
39 35.58
40 29.04
41 32.51
42 35.21
43 32.92
44 32.98
45 29.73
46 32.1
47 31.34
48 29.37
49 28.34
50 40.65
51 30.3
52 35.28
53 36.17
54 32.65
55 31.62
56 29.9
57 29.27
58 30.07
59 27.4
60 29.25
61 30.75
62 30.48
63 32.97
64 27.57
65 31.7
66 35.9
67 34.86
68 32.17


In [7]:
print(np.round(np.mean(np.array(psnrs)), 2), np.round(np.std(np.array(psnrs)), 2))

31.55 2.39
